# Emailing Participants with API

This notebook is used to email participants with their API keys. This is not to be used by participants themselves, but rather by the workshop organizers to send out keys.

However, you can take a look at the code to see how bulk e-mailing is done using `sendgrid` 

Sendgrid (from Twilio) is a service that allows you to send emails in bulk. It is a paid service, but it has a free tier that allows you to send up to 100 emails per day.
You can use it to send emails to participants with their API keys.

You can sign up for a free account at [SendGrid](https://sendgrid.com/).

There are other services that allow you to send emails in bulk, such as [Mailgun](https://www.mailgun.com/) and [Amazon SES](https://aws.amazon.com/ses/), but we will use SendGrid for this workshop.

In old days this was done using `smtplib` and `email` libraries, however, these days most e-mail providers have limits on how many emails you can send per day, so it is better to use a service that is designed for this purpose.
This notebook will show you how to use SendGrid to send emails to participants with their API keys

In [ ]:
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

# Store your key securely (e.g., using environment variable)
SENDGRID_API_KEY = os.getenv("SENDGRID_API")  # or paste directly (not recommended)
if not SENDGRID_API_KEY:
    raise ValueError("SENDGRID_API_KEY environment variable not set")
else:
    print("SendGrid API key is set.") # again we do not want to print the key itself!! then anyone could use it....
sender_email = "valdis.saulespurens@lnb.lv"
print("Sender email is set to:", sender_email)


## Reading e-mail participants from XLSX

Our participants are stored in an XLSX file, which we will read using `pandas`. We will then extract the email addresses and API keys from the DataFrame.



In [ ]:
# next lets load participants from emails.json in temp directory of our parent folder
from pathlib import Path
import pandas as pd
print(f"pandas version: {pd.__version__}")
emails_file = Path("../temp/BSSDH_2025_provisioned_keys.xlsx")
if not emails_file.exists():
    raise FileNotFoundError(f"Emails file {emails_file} does not exist.")
df = pd.read_excel(emails_file, engine='openpyxl')
print(f"Loaded {len(df)} participants from {emails_file}")
# shape
print(f"DataFrame shape: {df.shape}")
# columns
print(f"DataFrame columns: {df.columns.tolist()}")  
# show head 2 of first 3 columns
print("First 2 rows of the first 3 columns:")
display(df.iloc[:2, :3])

In [ ]:
# a slight complication is that E-mail column might actually contain multiple emails separated by commas or /
# our goal is to convert this dataframe into a list of dictionaries with following keys:
# 'emails' (list of emails (str)), 'name' (str), 'surname' (str), 'api_key' (str)
# so let's write a function that takes a row and returns a dictionary
import re
def row_to_dict(row):
    emails = emails = [e for e in re.split(r'[,\s/]+', row["E-mail"]) if e]
    return {
        'emails': emails,
        'name': row['Name'],
        'surname': row['Surname'],
        'api_key': row['api_key']
    }



In [ ]:
# now let's create that list of dictionaries
participants = df.apply(row_to_dict, axis=1).tolist()
print(f"Converted {len(participants)} participants to list of dictionaries.")

In [ ]:
# print those participants who have more than one email
# for participant in participants:
#     if len(participant['emails']) > 1:
#         print(f"Participant {participant['name']} {participant['surname']} has multiple emails: {participant['emails']}")

In [ ]:
import time
DELAY = 0.2  # delay in seconds to avoid hitting SendGrid rate limits
print(f"Sending emails with a delay of {DELAY} seconds just in case to avoid rate limits.")
for participant in participants:
    for email in participant['emails']:
        message = Mail(
            from_email=sender_email,
            to_emails=email,
            subject="Your Unique API Key for the BSSDH Workshop on August 7th",
            plain_text_content=(
                f"Dear {participant['name']},\n\n"
                f"Thank you for participating in our Using LLMs in Humanities Research via API workshop.\n\n"
                f"Your unique API key is:\n{participant['api_key']}\n\n"
                "Please keep this key secure and do not share it with others.\n\n"
                "This key is essential for actively participating in the workshop.\n"

                "The official repository for this workshop is available at:\n"
                "https://github.com/ValRCS/BSSDH_2025_workshop_LLM_API\n"
                "We will provide instructions at the workshop on how to access the repository and use the provided materials.\n\n"

                "See you on August 7th!\n\n"

                "According to the workshop schedule on: https://www.digitalhumanities.lv/bssdh/2025/Programme/\n"
                "The first session of this particular workshop will start at 11:30AM\n"
                "There is another workshop before ours which does not require this key\n\n"

                "There is no need to reply to this e-mail as everything will be explained at the workshop :) \n\n"
            
                "Best regards,\n"
                "On behalf of the BSSDH Workshop Team - Valdis Saulespurens\n"

            )
        )
        try:
            sg = SendGridAPIClient(SENDGRID_API_KEY)
            response = sg.send(message)
            print(f"✅ Sent to {email} – Status: {response.status_code}")
        except Exception as e:
            print(f"❌ Failed to send to {email}: {e}")
        time.sleep(DELAY) # just in case to avoid hitting SendGrid rate limits, which are supposed to be 10-20 emails per second

# SendGrid does not require explicit cleanup - however running Sendgrid from Notebook causes spinning wheel in VSCode
# TODO read more SendGrid documentation about cleanup
# URL: https://www.twilio.com/docs/sendgrid/for-developers/sending-email/quickstart-python
# so we will delete the SendGrid client object to close the connection - this is not strictly necessary
del sg
import gc
gc.collect()
print("All emails processed.")

